# Data fetching

In [73]:
import pandas as pd
import requests
 
df = pd.read_csv('full_city_list.txt', sep=";")
df_arg = df[df['Country']=='Argentina']
 
province_lst = []

for i in range(len(df_arg)):
    province_id = int(df_arg.iloc[i]['CityId'])
    url = 'https://worldweather.wmo.int/en/json/' + str(province_id) + '_en.json'
    response = requests.get(url)
    province_json = response.json()
    province_lst.append(province_json)
print(len(province_lst))

10


In [74]:
province_lst[0]

{'city': {'lang': 'en',
  'cityName': 'Bariloche',
  'cityLatitude': '-41.150000000',
  'cityLongitude': '-71.170000000',
  'cityId': 858,
  'isCapital': False,
  'stationName': 'Bariloche',
  'tourismURL': '',
  'tourismBoardName': '',
  'isDep': False,
  'timeZone': '-0300',
  'isDST': 'N',
  'member': {'memId': 50,
   'memName': 'Argentina',
   'shortMemName': '',
   'url': 'www.smn.gov.ar',
   'orgName': 'Servicio Meteorológico Nacional',
   'logo': 'argentina_logo.png',
   'ra': 3},
  'forecast': {'issueDate': '2025-04-20 17:32:00',
   'timeZone': 'Local',
   'forecastDay': [{'forecastDate': '2025-04-21',
     'wxdesc': '',
     'weather': 'Rain',
     'minTemp': '7',
     'maxTemp': '15',
     'minTempF': '45',
     'maxTempF': '59',
     'weatherIcon': 1401},
    {'forecastDate': '2025-04-22',
     'wxdesc': '',
     'weather': 'Rain',
     'minTemp': '5',
     'maxTemp': '14',
     'minTempF': '41',
     'maxTempF': '57',
     'weatherIcon': 1401},
    {'forecastDate': '2025-04

# Main Dataframe

In [75]:
# import pandas as pd 
# df = pd.read_json('Cordoba.json')
# df
import pandas as pd
df = pd.json_normalize(province_lst[0])


In [76]:
df

,city.lang,city.cityName,city.cityLatitude,city.cityLongitude,city.cityId,city.isCapital,city.stationName,city.tourismURL,city.tourismBoardName,city.isDep,...,city.climate.datab,city.climate.datae,city.climate.tempb,city.climate.tempe,city.climate.rdayb,city.climate.rdaye,city.climate.rainfallb,city.climate.rainfalle,city.climate.climatefromclino,city.climate.climateMonth
0,en,Bariloche,-41.150000000,-71.170000000,858,False,Bariloche,,,False,...,1981,2010,,,,,,,,"[{'month': 1, 'maxTemp': '22.6', 'minTemp': '6..."


In [77]:
df.columns

Index(['city.lang', 'city.cityName', 'city.cityLatitude', 'city.cityLongitude',
       'city.cityId', 'city.isCapital', 'city.stationName', 'city.tourismURL',
       'city.tourismBoardName', 'city.isDep', 'city.timeZone', 'city.isDST',
       'city.member.memId', 'city.member.memName', 'city.member.shortMemName',
       'city.member.url', 'city.member.orgName', 'city.member.logo',
       'city.member.ra', 'city.forecast.issueDate', 'city.forecast.timeZone',
       'city.forecast.forecastDay', 'city.climate.raintype',
       'city.climate.raindef', 'city.climate.rainunit', 'city.climate.datab',
       'city.climate.datae', 'city.climate.tempb', 'city.climate.tempe',
       'city.climate.rdayb', 'city.climate.rdaye', 'city.climate.rainfallb',
       'city.climate.rainfalle', 'city.climate.climatefromclino',
       'city.climate.climateMonth'],
      dtype='object')

In [78]:
df.columns

Index(['city.lang', 'city.cityName', 'city.cityLatitude', 'city.cityLongitude',
       'city.cityId', 'city.isCapital', 'city.stationName', 'city.tourismURL',
       'city.tourismBoardName', 'city.isDep', 'city.timeZone', 'city.isDST',
       'city.member.memId', 'city.member.memName', 'city.member.shortMemName',
       'city.member.url', 'city.member.orgName', 'city.member.logo',
       'city.member.ra', 'city.forecast.issueDate', 'city.forecast.timeZone',
       'city.forecast.forecastDay', 'city.climate.raintype',
       'city.climate.raindef', 'city.climate.rainunit', 'city.climate.datab',
       'city.climate.datae', 'city.climate.tempb', 'city.climate.tempe',
       'city.climate.rdayb', 'city.climate.rdaye', 'city.climate.rainfallb',
       'city.climate.rainfalle', 'city.climate.climatefromclino',
       'city.climate.climateMonth'],
      dtype='object')

# OBJECT SEGMENTATION

### 3 objects -> [city,member] ; [forecast]; [climate]

In [79]:
import pandas as pd
from pandas import json_normalize

mem_cols = ['city.lang', 'city.cityName', 'city.cityLatitude', 'city.cityLongitude',
    'city.cityId', 'city.isCapital', 'city.stationName', 'city.tourismURL',
    'city.tourismBoardName', 'city.isDep', 'city.timeZone', 'city.isDST',
    'city.member.memId', 'city.member.memName', 'city.member.shortMemName',
    'city.member.url', 'city.member.orgName', 'city.member.logo',
    'city.member.ra']

city_member = df[mem_cols]
city_member


,city.lang,city.cityName,city.cityLatitude,city.cityLongitude,city.cityId,city.isCapital,city.stationName,city.tourismURL,city.tourismBoardName,city.isDep,city.timeZone,city.isDST,city.member.memId,city.member.memName,city.member.shortMemName,city.member.url,city.member.orgName,city.member.logo,city.member.ra
0,en,Bariloche,-41.150000000,-71.170000000,858,False,Bariloche,,,False,-0300,N,50,Argentina,,www.smn.gov.ar,Servicio Meteorológico Nacional,argentina_logo.png,3


In [80]:
import pandas as pd
from pandas import json_normalize

# Define the columns to keep (excluding the nested column to normalize)
climate_cols = [
    'city.climate.raintype', 'city.climate.raindef', 'city.climate.rainunit',
    'city.climate.datab', 'city.climate.datae', 'city.climate.tempb', 'city.climate.tempe',
    'city.climate.rdayb', 'city.climate.rdaye', 'city.climate.rainfallb', 'city.climate.rainfalle',
    'city.climate.climatefromclino'
]

changes = ['city.climate.climateMonth']  # nested column to normalize

# Extract non-nested columns
df_base = df[climate_cols].copy().reset_index(drop=True)

# Normalize nested column
normalized_parts = []
for col in changes:
    nested_data = df[col]
    exploded = nested_data.explode().reset_index(drop=True)
    norm = json_normalize(exploded)
    norm.columns = [f"{col}.{sub}" for sub in norm.columns]
    normalized_parts.append(norm)

# Combine base + normalized
city_climate = pd.concat([df_base] + normalized_parts, axis=1)

# Show result
city_climate


,city.climate.raintype,city.climate.raindef,city.climate.rainunit,city.climate.datab,city.climate.datae,city.climate.tempb,city.climate.tempe,city.climate.rdayb,city.climate.rdaye,city.climate.rainfallb,...,city.climate.climateMonth.month,city.climate.climateMonth.maxTemp,city.climate.climateMonth.minTemp,city.climate.climateMonth.meanTemp,city.climate.climateMonth.maxTempF,city.climate.climateMonth.minTempF,city.climate.climateMonth.meanTempF,city.climate.climateMonth.raindays,city.climate.climateMonth.rainfall,city.climate.climateMonth.climateFromMemDate
0,Rainfall,0.1,mm,1981.0,2010.0,,,,,,...,1,22.6,6.6,None,72.7,43.9,None,3.7,20.0,2002-11-15
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,23.0,6.3,None,73.4,43.3,None,3.4,15.3,2002-11-15
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3,19.7,4.7,None,67.5,40.5,None,6.7,35.2,2002-11-15
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4,14.6,2.3,None,58.3,36.1,None,8.2,58.9,2002-11-15
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5,10.2,0.6,None,50.4,33.1,None,13.6,130.1,2002-11-15
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6,6.9,-0.6,None,44.4,30.9,None,16.1,170.1,2002-11-15
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7,6.7,-1.6,None,44.1,29.1,None,14.0,122.1,2002-11-15
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8,8.3,-1.1,None,46.9,30.0,None,13.0,101.1,2002-11-15
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9,11.5,-0.2,None,52.7,31.6,None,9.1,56.5,2002-11-15
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10,14.8,1.8,None,58.6,35.2,None,8.3,49.4,2002-11-15


In [88]:
import pandas as pd
from pandas import json_normalize

# Define forecast columns
forecast_cols = ['city.forecast.issueDate', 'city.forecast.timeZone']
nested_col = 'city.forecast.forecastDay'

# Extract base columns (excluding nested list)
df_base = df[forecast_cols].copy().reset_index(drop=True)

# Explode and normalize nested forecast list (assuming it's a list of dicts)
exploded = df[nested_col].apply(pd.Series).stack().reset_index(level=1, drop=True)
exploded = exploded.reset_index(drop=True)

# Normalize each forecast entry
forecast_normalized = json_normalize(exploded)

# Add a label: Today, Today+1, ..., Today+6
forecast_normalized['ForecastDay'] = ['Today+{}'.format(i) for i in range(len(forecast_normalized))]

# If your original df has multiple cities, repeat each city’s info for 7 rows
df_repeated = pd.concat([df_base.loc[i].repeat(7) for i in df_base.index], ignore_index=True)

# Combine everything
city_forecast = pd.concat([df_repeated.reset_index(drop=True), forecast_normalized], axis=1)

# Reorder columns if needed
city_forecast = city_forecast[['ForecastDay', 'forecastDate', 'wxdesc', 'weather', 'minTemp', 'maxTemp', 'minTempF', 'maxTempF', 'weatherIcon']]

city_forecast = city_forecast.head(6)

# Show result
city_forecast


,ForecastDay,forecastDate,wxdesc,weather,minTemp,maxTemp,minTempF,maxTempF,weatherIcon
0,Today+0,2025-04-21,,Rain,7,15,45,59,1401.0
1,Today+1,2025-04-22,,Rain,5,14,41,57,1401.0
2,Today+2,2025-04-23,,Partly Cloudy,6,14,43,57,2202.0
3,Today+3,2025-04-24,,Light Rain,4,12,39,54,1502.0
4,Today+4,2025-04-25,,Light Rain,3,12,37,54,1502.0
5,Today+5,2025-04-26,,Mostly Cloudy,2,8,36,46,2302.0


In [86]:
city_forecast.columns

Index([             0, 'forecastDate',       'wxdesc',      'weather',
            'minTemp',      'maxTemp',     'minTempF',     'maxTempF',
        'weatherIcon',  'ForecastDay'],
      dtype='object')

# 

# rutujas code 